In [11]:
import joblib
import pickle
import torch

#Basma
# classifer = joblib.load("C:/Users/basma/Fourth Year/Graduation_Project/squeezenetfinetuned.pkl")

#Bassem
classifer = joblib.load("squeezenetuntrained.pkl")

In [12]:
# loaded_model = pickle.load("squeezenetfinetuned.pkl")
import sklearn 
import numpy as np

model = pickle.load(open('squeezenetuntrained.pkl', 'rb'))
model.eval()

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [14]:
# Segmentation then Cropping algorithm 
import cv2
from imutils import contours
from skimage.filters import threshold_otsu

# Load image, grayscale, Otsu's threshold
image_array = []
results = []
# image = cv2.imread('C:/Users/basma/Fourth Year/Graduation_Project/O001 (1).png')
# image_array.append(image)
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
# inv = cv2.bitwise_not(thresh)
# rgb = cv2.cvtColor(inv, cv2.COLOR_GRAY2RGB)

#---------------------------------------------------------------------
def Otsu(myImage):
  sample_image = cv2.imread(myImage)
  img = cv2.cvtColor(sample_image,cv2.COLOR_BGR2RGB)

  # plt.axis('off')
  # plt.imshow(img)

  img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

  thresh = threshold_otsu(img_gray)
  img_otsu  = img_gray < thresh

  # plt.imshow(img_otsu)

  filtered = filter_image(img, img_otsu)
  filteredBW=cv2.cvtColor(filtered,cv2.COLOR_RGB2GRAY)
  return filteredBW



def filter_image(image, mask):

    r = image[:,:,0] * mask
    g = image[:,:,1] * mask
    b = image[:,:,2] * mask

    return np.dstack([r,g,b])

image = Otsu('C:/Users/basma/Fourth Year/Graduation_Project/D021.png')

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
# inv = cv2.bitwise_not(thresh)

cv2.imwrite('image2.png', image)
#---------------------------------------------------------------------

# Find contours, sort from left-to-right, then crop
# cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# cnts, _ = contours.sort_contours(cnts, method="left-to-right")

# ROI_number = 0
# for c in cnts:
#     area = cv2.contourArea(c)
#     if area > 50:
#         x,y,w,h = cv2.boundingRect(c)
#         ROI = 255 - image[y:y+h, x:x+w]
#         img1 = cv2.copyMakeBorder(ROI, 20,20,20,20,cv2.BORDER_CONSTANT, value=[0,0,0])
#         cv2.imwrite('ROIVVV_{}.png'.format(ROI_number), img1)
        
#         # image_array.append(img1)
#         cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
#         ROI_number += 1



True

In [15]:
import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile
import cv2

#Basma
input_image = Image.open('image2.png').convert('RGB')
# input_image = Image.open('ROIVVV_0.png').convert('RGB')
# input_image = Image.open('C:/Users/basma/Fourth Year/Graduation_Project/SegmentedDatasetAgain/train/A040/A040_1.png').convert('RGB')
#Bassem
# input_image = Image.open('D:/Ahmed Bassem/MIU/Year 4/Graduation Project/Datasets/SegmentedDataset/SegmentedDataset/test/O001/O001_1.png').convert('RGB')

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # transforms.Pad(50),
])
input_tensor = preprocess(input_image)
# input_tensor = preprocess(Image.fromarray(i))
input_batch = input_tensor.unsqueeze(0)
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

# Tensor of shape 1000, with confidence scores over Aegyptos' 1072 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Read the categories
#Basma
with open("classes_names.txt", "r") as f:

#Bassem
# with open("D:/Ahmed Bassem/MIU\Year 4/Graduation Project/classes_names.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

top1 = categories[top5_catid[0]]
print(top1)
results.append(top1)

tensor([19.2779,  3.9257,  7.1866,  ...,  1.0307,  0.5112,  0.8537])
D021 0.9993090629577637
S010 0.00026815372984856367
S002 0.0001975266495719552
D022 8.038392115850002e-05
D048A 7.427437230944633e-05
D021


In [6]:
# im = Image.fromarray(image_array[0])
# im.show()
cv2.imshow("test",image_array[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: list index out of range

In [7]:
# # Read the categories

# #Basma
# # with open("C:/Users/basma/Fourth Year/Graduation_Project/classes_names.txt", "r") as f:

# #Bassem
# with open("D:/Ahmed Bassem/MIU\Year 4/Graduation Project/classes_names.txt", "r") as f:
#     categories = [s.strip() for s in f.readlines()]
# # Show top categories per image
# top5_prob, top5_catid = torch.topk(probabilities, 5)
# for i in range(top5_prob.size(0)):
#     print(categories[top5_catid[i]], top5_prob[i].item())

# top1 = categories[top5_catid[0]]
# print(top1)

A040 0.8683706521987915
A040A 0.08197934925556183
A049 0.03442122042179108
C003 0.00968263577669859
A041 0.0015775265637785196
A040


In [15]:
import matching_algorithm as ma
print(ma.dictionary_matching(top1))


ValueError: 'AA029' is not in list

In [82]:
import googletrans
from googletrans import *

In [83]:
match = ma.dictionary_matching(top1)

In [84]:
translator = googletrans.Translator()
translate=translator.translate(match,dest='arabic')
print(translate.text)

منزل
